# Understanding asyncio
Let us first do some simple tests on getting OHLCs using synchronous and async methods.

In [1]:
import asyncio
import pickle
import pandas as pd

from collections import defaultdict

In [2]:
from ib_insync import *

In [3]:
import nest_asyncio
nest_asyncio.apply()
util.startLoop()

In [4]:
MARKET = 'SNP'

HOST = '127.0.0.1'
PORT = 4002 if MARKET.upper() == 'NSE' else 4002  # Paper trades!!
CID = 0

DATAPATH = './data/'
LONG_BAR_FORMAT = '{desc:<25}{percentage:3.0f}%|{bar:30}{r_bar}'

In [5]:
## Get the contracts
with open('./data/und_cts.pkl', 'rb') as f:
    sym_con = pickle.load(f)
all_cts = list(sym_con.values())
stock_cts = [c for c in all_cts if isinstance(c, Stock)]

# Test ohlc coro only

In [15]:
async def ohlcCoro(c, DURATION=365, SLEEP=5):
    ohlc = ib.reqHistoricalDataAsync(
        contract=c,
        endDateTime="",
        durationStr=str(DURATION) + ' D',
        barSizeSetting="1 day",
        whatToShow="Trades",
        useRTH=True)

    # Refer: https://stackoverflow.com/a/40572169/7978112
    await asyncio.sleep(SLEEP)

    return {c.symbol: ohlc}

In [58]:
cts = list(all_cts)[:50]

In [71]:
%%time
if __name__ == "__main__":

    async def get_ohlcs(cts):
        tasks = [asyncio.create_task(
            ohlcCoro(c, DURATION=365, SLEEP=5), name=c.symbol) for c in cts]
    
        result = defaultdict(dict)
        done, pending = await asyncio.wait(tasks)

        '''for task in tasks:
            if task in done:
                result.update({task.get_name(): task.result()})'''
        for task in done:
            result.update({task.get_name(): type(list(task.result().values())[0])})
                
        return result, tasks

    with IB().connect(HOST, PORT, CID) as ib:
        result = ib.run(get_ohlcs(cts))

Wall time: 5.03 s


In [72]:
result

(defaultdict(dict,
             {'AMT': asyncio.futures.Future,
              'BK': asyncio.futures.Future,
              'BIIB': asyncio.futures.Future,
              'AIG': asyncio.futures.Future,
              'ADBE': asyncio.futures.Future,
              'ACN': asyncio.futures.Future,
              'ABT': asyncio.futures.Future,
              'ABBV': asyncio.futures.Future,
              'SPX': asyncio.futures.Future,
              'AAPL': asyncio.futures.Future,
              'XEO': asyncio.futures.Future,
              'VIX': asyncio.futures.Future,
              'MXEA': asyncio.futures.Future,
              'MXEF': asyncio.futures.Future,
              'XSP': asyncio.futures.Future,
              'OEX': asyncio.futures.Future,
              'AMZN': asyncio.futures.Future,
              'AMGN': asyncio.futures.Future,
              'BA': asyncio.futures.Future,
              'AXP': asyncio.futures.Future,
              'BAC': asyncio.futures.Future,
              'BKNG': asyncio.

In [45]:
if __name__ == "__main__":

    async def get_ohlcs(cts):
        tasks = [asyncio.create_task(
            ohlcCoro(c, DURATION=5, SLEEP=5), name=c.symbol) for c in cts]

        result = dict()
        for coro in asyncio.as_completed(tasks, timeout=None):
            result.update({coro: await coro})

        return result

    with IB().connect(HOST, PORT, CID) as ib:
        result = ib.run(get_ohlcs(cts))

In [46]:
result

{<coroutine object as_completed.<locals>._wait_for_one at 0x0ADC51A8>: {'DJX': <Future finished result=[BarData(date...rCount=10644)]>},
 <coroutine object as_completed.<locals>._wait_for_one at 0x0ADC53E8>: {'SPX': <Future finished result=[BarData(date...arCount=4548)]>},
 <coroutine object as_completed.<locals>._wait_for_one at 0x0ADC59E8>: {'OEX': <Future finished result=[BarData(date...arCount=6947)]>},
 <coroutine object as_completed.<locals>._wait_for_one at 0x0ADC5968>: {'MXEF': <Future finished result=[BarData(date...arCount=1087)]>},
 <coroutine object as_completed.<locals>._wait_for_one at 0x0ADC5A28>: {'XEO': <Future finished result=[BarData(date...rCount=14468)]>},
 <coroutine object as_completed.<locals>._wait_for_one at 0x0ADC5AE8>: {'MXEA': <Future finished result=[BarData(date...arCount=1352)]>},
 <coroutine object as_completed.<locals>._wait_for_one at 0x0ADC5A68>: {'VIX': <Future finished result=[BarData(date...barCount=639)]>},
 <coroutine object as_completed.<locals

In [47]:
list(list(result.values())[0].values())[0].result()

[BarData(date=datetime.date(2020, 7, 28), open=265.29, high=265.57, low=263.62, close=263.79, volume=0, average=0.0, barCount=10116),
 BarData(date=datetime.date(2020, 7, 29), open=263.88, high=266.02, low=263.75, close=265.4, volume=0, average=0.0, barCount=9822),
 BarData(date=datetime.date(2020, 7, 30), open=263.67, high=263.75, low=259.92, close=263.14, volume=0, average=0.0, barCount=10921),
 BarData(date=datetime.date(2020, 7, 31), open=264.09, high=264.4, low=260.14, close=264.28, volume=0, average=0.0, barCount=12439),
 BarData(date=datetime.date(2020, 8, 3), open=265.42, high=267.07, low=265.34, close=266.64, volume=0, average=0.0, barCount=10644)]

In [ ]:
## Build base async functions with its component async functions
async def ohlcAsync(c, duration=365):
    '''A simple async for ohlc'''
    ohlc_coro = ib.reqHistoricalDataAsync(
        contract=c,
        endDateTime="",
        durationStr=str(duration) + ' D',
        barSizeSetting="1 day",
        whatToShow="Trades",
        useRTH=True)
    return ohlc_coro

async def chainsAsync(c):
    chain_coro = ib.reqSecDefOptParamsAsync(
        underlyingSymbol=c.symbol, futFopExchange="", 
        underlyingSecType=c.secType, underlyingConId=c.conId)
    return chain_coro

async def mdataAsync(c, FILL_DELAY=5):
    tick_coro = ib.reqMktData(
        c, '456, 104, 106, 100, 101, 165')
    await asyncio.sleep(FILL_DELAY)
    ib.cancelMktData(c)
    
    return tick_coro

async def priceAsync(c, DELAY=5):
    price_coro = await ib.reqTickersAsync(c)
    await asyncio.sleep(DELAY)
    return price_coro

async def baseAsync(c):
    tasks = [ohlcAsync(c), chainsAsync(c), mdataAsync(c), priceAsync(c)]
    return await asyncio.gather(*tasks)

In [ ]:
import random
random.seed(1234)
cts = random.sample(all_cts, 5)
cts

In [ ]:
%%time
## Extract the base
with IB().connect(HOST, PORT, CID) as ib:
    base = {c.symbol: ib.run(baseAsync(c)) for c in all_cts}
#     base = {c.symbol: ib.run(baseAsync(c)) for c in cts} # !!! DATA IS LIMITED !!! not working !!!!
print(base)

In [ ]:
base['KMI']

In [ ]:
## Assemble the base dfs
ohlcs = defaultdict(dict)
mdata = defaultdict(dict)
chains = defaultdict(dict)
undPrices = defaultdict(dict)

for k, v in base.items():
    ohlcs.update({k: v[0].result() if v[0].done() else None})
    chains.update({k: v[1].result() if v[1].done() else None})
    mdata.update({k: v[2]})
    undPrices.update({k: v[3][0]})

In [ ]:
undPrices

In [ ]:
# cleanup the ohlcs
ohlc_dfs = []
mdata_dfs = []

for k, v in ohlcs.items():
    
    if v: # it is not None
        
        # for ohlcs
        df1 = util.df(v)
        df1.insert(0, 'symbol', k)
        ohlc_dfs.append(df1)
        
        # for market data
        m_df = pd.DataFrame(util.df([mdata[k]]))
        div_df = pd.DataFrame(m_df.dividends.tolist())

        df2 = m_df.drop('dividends', 1).join(div_df)
        df2.insert(0, 'symbol', k)
        mdata_dfs.append(df2)

In [ ]:
df_ohlcs = pd.concat(ohlc_dfs, ignore_index=True)
df_unds = pd.concat(mdata_dfs, ignore_index=True)

In [ ]:
# ... set up the final mkt df columns
cols = ['symbol', 'time', 'low13week', 'high13week', 'low26week',
        'high26week', 'low52week', 'high52week',  'putVolume', 'callVolume',
        'past12Months', 'next12Months', 'nextDate', 'nextAmount',
        'avOptionVolume', 'histVolatility', 'impliedVolatility']

In [ ]:
df_unds[cols]

The above works beautifully for stock ohlcs. But we have some problems and challenges: 

1. We can try to replicate the success to other coros
2. We want to build a dictionary with sym:ohlc to identify which ohlcs belong to what symbol
3. It doesn't work for index ohlcs. We need a way for it to wait for a certain time and quit, if it is not getting.